# Ising Model

## construct a 3-qubit state

In [1]:
from qiskit.quantum_info import Statevector
import numpy as np
import pylatexenc

zero = Statevector([1,0])
print("zero	is", zero)
one = Statevector([0,1])

psi = one.tensor(zero.tensor(zero))
print('psi is', psi)

psi = one^zero^zero
psi.draw("latex")

psi = 1/np.sqrt(2) * (zero^zero^zero) + 1/np.sqrt(2) * (one^one^one)
psi.draw("latex")

zero	is Statevector([1.+0.j, 0.+0.j],
            dims=(2,))
psi is Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


<IPython.core.display.Latex object>

## Construct a Pauli state

In [2]:
from qiskit.quantum_info import Pauli

In [3]:
z0z1 = Pauli("ZZI")
print("z0z1 is", z0z1)
print('and its matrix is')
print(z0z1.to_matrix())

z0z1 is ZZI
and its matrix is
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j]]


In [4]:
print('The sparse representation of z0z1 is')
print(z0z1.to_matrix(sparse=True))

The sparse representation of z0z1 is
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 8 stored elements and shape (8, 8)>
  Coords	Values
  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(-1+0j)
  (3, 3)	(-1+0j)
  (4, 4)	(-1+0j)
  (5, 5)	(-1+0j)
  (6, 6)	(1+0j)
  (7, 7)	(1+0j)


## Operate a Pauli state like Z or I

In [5]:
from qiskit.quantum_info import SparsePauliOp as PauliOp

In [6]:
H_cut = PauliOp(Pauli("ZZI")) + PauliOp(Pauli("ZIZ"))
print('H_cut is')
print(H_cut)
print('The sparse representation of H_cut is')
print(H_cut.to_matrix(sparse=True))

H_cut is
SparsePauliOp(['ZZI', 'ZIZ'],
              coeffs=[1.+0.j, 1.+0.j])
The sparse representation of H_cut is
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 8 stored elements and shape (8, 8)>
  Coords	Values
  (0, 0)	(2+0j)
  (1, 1)	0j
  (2, 2)	0j
  (3, 3)	(-2+0j)
  (4, 4)	(-2+0j)
  (5, 5)	0j
  (6, 6)	0j
  (7, 7)	(2+0j)


## Construct a linear combination of Hamiltonian

In [ ]:
H_Ising = (
    -0.5*PauliOp('ZZI') + 
    2*PauliOp('ZIZ') - 
    PauliOp('IZZ') + 
    PauliOp('IZI') - 
    5*PauliOp('IIZ')
)

psi = one^zero^zero
E = psi.expectation_value(H_cut)
print(E)

(-2+0j)
